In [ ]:
!pip install pandas numpy torch sentence-transformers transformers pillow tqdm

In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import requests
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
from io import BytesIO
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from transformers import CLIPTextModel, CLIPTokenizer
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel


In [ ]:
df = pd.read_csv("train.csv", quotechar='"', doublequote=True, escapechar='\\')
df = df.iloc[10000:35000,:] # This line is no longer needed after adding nrows
print(df.head())

       sample_id                                    catalog_content  \
10000     144360  Item Name: Oak Smoke Decaf Black Tea (Loose) (...   
10001      31120  Item Name: International Collection Virgin Fla...   
10002     239791  Item Name: Warheads Limited Edition Halloween ...   
10003      90802  Item Name: Swad Cardamom Seed, 14 Ounce\nBulle...   
10004     114112  Item Name: Ruffles Potato Chips, Lime & Jalape...   

                                              image_link  price  
10000  https://m.media-amazon.com/images/I/81GdsREBZw...  27.40  
10001  https://m.media-amazon.com/images/I/813xJZ-8Z5...  12.49  
10002  https://m.media-amazon.com/images/I/81zerWnK-A...  12.99  
10003  https://m.media-amazon.com/images/I/91UA80t+nK...  25.99  
10004  https://m.media-amazon.com/images/I/71l0W92MCB...   4.79  


In [ ]:
import re

def clean_text(text):
    if pd.isna(text): return ""
    text = re.sub(r'\s+', ' ', text)                # collapse whitespace
    text = re.sub(r'http\S+', '', text)             # remove URLs
    text = text.replace("•", "").replace("–", "-")
    return text.strip()

def extract_value_unit(text):
    value = re.search(r'Value:\s*([\d.]+)', text)
    unit = re.search(r'Unit:\s*([\w]+)', text)
    return float(value.group(1)) if value else np.nan, unit.group(1).lower() if unit else "unknown"

df["clean_text"] = df["catalog_content"].apply(clean_text)
df[["value", "unit"]] = df["catalog_content"].apply(lambda x: pd.Series(extract_value_unit(x)))


In [ ]:
df

,sample_id,catalog_content,image_link,price,clean_text,value,unit
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,fl
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Item Name: Salerno Cookies, The Original Butte...",32.00,ounce
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,ounce
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,ounce
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,count
...,...,...,...,...,...,...,...
9995,245122,"Item Name: Flock, Chicken Skin Crisps Hattie B...",https://m.media-amazon.com/images/I/71jSwzszMW...,6.29,"Item Name: Flock, Chicken Skin Crisps Hattie B...",2.50,ounce
9996,211102,Item Name: NY Spice Shop Sanded Cinnamon Drops...,https://m.media-amazon.com/images/I/71RgY-MIaz...,10.49,Item Name: NY Spice Shop Sanded Cinnamon Drops...,16.00,ounce
9997,124577,Item Name: tag Vintage Santa Face with Wreath ...,https://m.media-amazon.com/images/I/61HoEdWE5l...,22.99,Item Name: tag Vintage Santa Face with Wreath ...,1.00,count
9998,203821,"Item Name: Marshalls Creek Spices, Select CARA...",https://m.media-amazon.com/images/I/81QfDLpkQO...,11.95,"Item Name: Marshalls Creek Spices, Select CARA...",8.00,ounce


In [ ]:
# SBERT / MiniLM
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# CLIP text encoder
clip_model_name = "openai/clip-vit-base-patch32"
clip_tokenizer = CLIPTokenizer.from_pretrained(clip_model_name)
clip_model = CLIPTextModel.from_pretrained(clip_model_name)
clip_model.eval()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [ ]:
clip_embs = []
sbert_embs = []

for text in tqdm(df["clean_text"], desc="Encoding text"):
    # ----- SBERT embedding -----
    sbert_emb = sbert_model.encode(text, normalize_embeddings=True)
    sbert_embs.append(sbert_emb)

    # ----- CLIP embedding -----
    inputs = clip_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=77)
    with torch.no_grad():
        clip_output = clip_model(**inputs)
        clip_emb = clip_output.last_hidden_state[:,0,:]  # take [CLS] token embedding
        clip_emb = clip_emb / clip_emb.norm(dim=-1, keepdim=True)
        clip_embs.append(clip_emb.squeeze().cpu().numpy())



# Optional: combine embeddings (concatenate)
df["text_emb_combined"] = [np.concatenate([s, c]) for s, c in zip(sbert_embs, clip_embs)]

print("✅ Text embeddings ready")



Streaming output truncated to the last 5000 lines.
Encoding text: 100%|██████████| 25000/25000 [1:45:25<00:00,  3.95it/s]


✅ Text embeddings ready


In [ ]:
print("Combined dim:", len(df['text_emb_combined'][0]))
df.to_pickle("text_image_embeddings2.pkl")

KeyError: 0

In [ ]:
df

,sample_id,catalog_content,image_link,price,clean_text,value,unit,text_emb_combined
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,fl,"[0.03326046, 0.01084006, -0.056023084, 0.08874..."
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Item Name: Salerno Cookies, The Original Butte...",32.00,ounce,"[-0.042282123, 0.03143435, -0.04748076, 0.0682..."
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,ounce,"[-0.03363379, 0.0030081016, 0.0166668, 0.08349..."
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,ounce,"[-0.10701239, -0.07945337, 0.012586073, -0.057..."
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,count,"[0.054222934, 0.07242486, -0.043134164, 0.0518..."
...,...,...,...,...,...,...,...,...
9995,245122,"Item Name: Flock, Chicken Skin Crisps Hattie B...",https://m.media-amazon.com/images/I/71jSwzszMW...,6.29,"Item Name: Flock, Chicken Skin Crisps Hattie B...",2.50,ounce,"[-0.0052951975, 0.01747233, -0.020774389, 0.10..."
9996,211102,Item Name: NY Spice Shop Sanded Cinnamon Drops...,https://m.media-amazon.com/images/I/71RgY-MIaz...,10.49,Item Name: NY Spice Shop Sanded Cinnamon Drops...,16.00,ounce,"[-0.114072844, -0.056503575, 0.06565911, 0.037..."
9997,124577,Item Name: tag Vintage Santa Face with Wreath ...,https://m.media-amazon.com/images/I/61HoEdWE5l...,22.99,Item Name: tag Vintage Santa Face with Wreath ...,1.00,count,"[-0.0062542404, 0.11907307, -0.0043826094, 0.0..."
9998,203821,"Item Name: Marshalls Creek Spices, Select CARA...",https://m.media-amazon.com/images/I/81QfDLpkQO...,11.95,"Item Name: Marshalls Creek Spices, Select CARA...",8.00,ounce,"[-0.07150947, -0.0832567, 0.0011477125, 0.0575..."


In [ ]:




# ------------------------
# Load Models
# ------------------------
# CLIP
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

# ResNet50 (ImageNet)
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet = nn.Sequential(*list(resnet.children())[:-1])  # remove final FC
resnet.eval()

# ImageNet preprocessing
imagenet_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

# ------------------------
# Helper Functions
# ------------------------
def get_image_from_url(url):
    """Download image into memory and return PIL.Image object."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content)).convert("RGB")
            return image
    except Exception:
        pass
    return None

def get_clip_embedding(image):
    """Compute CLIP image embedding."""
    try:
        inputs = clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            emb = clip_model.get_image_features(**inputs)
        emb = emb / emb.norm(dim=-1, keepdim=True)
        return emb.squeeze().cpu().numpy()
    except Exception:
        return np.zeros(512)

def get_imagenet_embedding(image):
    """Compute ImageNet (ResNet50) embedding."""
    try:
        img_tensor = imagenet_transform(image).unsqueeze(0)
        with torch.no_grad():
            emb = resnet(img_tensor)
        emb = emb.squeeze().cpu().numpy()
        emb = emb / np.linalg.norm(emb)
        return emb
    except Exception:
        return np.zeros(2048)

# ------------------------
# Process Images from URLs
# ------------------------
from io import BytesIO

clip_embs, imagenet_embs, combined_embs = [], [], []

for url in tqdm(df["image_link"], desc="Encoding images"):
    image = get_image_from_url(url)
    if image:
        clip_emb = get_clip_embedding(image)
        imgnet_emb = get_imagenet_embedding(image)
        combined_emb = np.concatenate([clip_emb, imgnet_emb])
    else:
        clip_emb = np.zeros(512)
        imgnet_emb = np.zeros(2048)
        combined_emb = np.zeros(2560)

    clip_embs.append(clip_emb)
    imagenet_embs.append(imgnet_emb)
    combined_embs.append(combined_emb)

# Add embeddings to DataFrame
df["clip_emb"] = clip_embs
df["imagenet_emb"] = imagenet_embs
df["img_emb_combined"] = combined_embs

print("✅ Image embeddings ready")



In [ ]:
df.to_pickle("text_image_embeddings2.pkl")

In [ ]:
df.to_csv("text_image_embedings_train.csv")

In [ ]:
df.to_csv("text_image_embedings_train.csv")

In [ ]:
df


,sample_id,catalog_content,image_link,price,clean_text,value,unit,text_emb_combined,clip_emb,imagenet_emb,img_emb_combined
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,fl,"[0.03326046, 0.01084006, -0.056023084, 0.08874...","[0.009212236, 0.0026213306, -0.030116022, -0.0...","[0.03180872, 0.054117072, 0.0008719487, 0.0015...","[0.009212236, 0.0026213306, -0.030116022, -0.0..."
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Item Name: Salerno Cookies, The Original Butte...",32.00,ounce,"[-0.042282123, 0.03143435, -0.04748076, 0.0682...","[0.014391732, -0.049200386, 0.011009298, 0.054...","[0.0049155937, 0.06572621, 4.4906494e-05, 0.00...","[0.014391732, -0.049200386, 0.011009298, 0.054..."
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,ounce,"[-0.03363379, 0.0030081016, 0.0166668, 0.08349...","[-0.010649164, -0.038380288, -0.009885343, 0.0...","[0.025890373, 0.021702567, 0.010921363, 0.0052...","[-0.010649164, -0.038380288, -0.009885343, 0.0..."
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,ounce,"[-0.10701239, -0.07945337, 0.012586073, -0.057...","[0.058353223, -0.004353989, 0.049397968, 0.017...","[0.028660946, 0.041502114, 0.021437678, 0.0006...","[0.058353223, -0.004353989, 0.049397968, 0.017..."
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,count,"[0.054222934, 0.07242486, -0.043134164, 0.0518...","[-0.035436884, 0.009175435, -0.022357896, -0.0...","[0.01593395, 0.060073067, 0.017688613, 0.00177...","[-0.035436884, 0.009175435, -0.022357896, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,245122,"Item Name: Flock, Chicken Skin Crisps Hattie B...",https://m.media-amazon.com/images/I/71jSwzszMW...,6.29,"Item Name: Flock, Chicken Skin Crisps Hattie B...",2.50,ounce,"[-0.0052951975, 0.01747233, -0.020774389, 0.10...","[0.022693401, -0.037833117, 0.015477225, 0.042...","[0.00941888, 0.059758104, 0.011924964, 0.00099...","[0.022693401, -0.037833117, 0.015477225, 0.042..."
9996,211102,Item Name: NY Spice Shop Sanded Cinnamon Drops...,https://m.media-amazon.com/images/I/71RgY-MIaz...,10.49,Item Name: NY Spice Shop Sanded Cinnamon Drops...,16.00,ounce,"[-0.114072844, -0.056503575, 0.06565911, 0.037...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0...","[0.01598184, 0.047277354, 0.009380544, 0.00158...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0..."
9997,124577,Item Name: tag Vintage Santa Face with Wreath ...,https://m.media-amazon.com/images/I/61HoEdWE5l...,22.99,Item Name: tag Vintage Santa Face with Wreath ...,1.00,count,"[-0.0062542404, 0.11907307, -0.0043826094, 0.0...","[0.019545663, 0.019494476, 0.032799203, 0.0371...","[0.010791587, 0.01188077, 0.026226072, 0.01902...","[0.019545663, 0.019494476, 0.032799203, 0.0371..."
9998,203821,"Item Name: Marshalls Creek Spices, Select CARA...",https://m.media-amazon.com/images/I/81QfDLpkQO...,11.95,"Item Name: Marshalls Creek Spices, Select CARA...",8.00,ounce,"[-0.07150947, -0.0832567, 0.0011477125, 0.0575...","[0.008147497, -0.02495418, 0.016679136, 0.0238...","[0.02273915, 0.071735054, 0.00043513643, 0.000...","[0.008147497, -0.02495418, 0.016679136, 0.0238..."


In [ ]:
df.drop(['imagenet_emb'],axis=1,inplace=True)

In [ ]:
df["imagenet_emb"] = imagenet_embs

In [ ]:
df.to_pickle("text_image_embeddings.pkl")

In [ ]:

X = np.vstack(df["img_emb_combined"].values)
print("Shape:", X.shape)   # (num_items, 512)
pca = PCA(n_components=512, random_state=42)
X_reduced = pca.fit_transform(X)
df["img_emb_combined_reduced"] = list(X_reduced)
print("Shape:", X_reduced.shape)

In [ ]:

n_clusters = 15# tune this
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_reduced)

# df["cluster"] = labels




In [ ]:

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_vis = tsne.fit_transform(X_reduced)

plt.figure(figsize=(8,6))
plt.scatter(X_vis[:,0], X_vis[:,1], c=labels, cmap='tab10', s=10)
plt.title("Image Embedding Clusters")
plt.show()


In [ ]:
print(labels)
df["cluster"] = labels

In [ ]:


def apply_pca_to_cluster(group, n_components=300):
    """
    Apply PCA to all embeddings in a single cluster.
    """
    X = np.vstack(group["img_emb_combined_reduced"].values)
    pca = PCA(n_components=n_components, random_state=42)
    X_reduced = pca.fit_transform(X)
    group["img_emb_pca"] = list(X_reduced)
    return group


In [ ]:
df_new = df.groupby("cluster", group_keys=False).apply(apply_pca_to_cluster)


In [ ]:
print(df_new[["cluster", "img_emb_pca"]].head())

# Example: check one cluster's shape
X0 = np.vstack(df_new[df_new["cluster"] == 0]["img_emb_pca"])
print("Cluster 0 PCA shape:", X0.shape)


In [ ]:
X = np.vstack(df["text_emb_combined"].values)
print("Shape:", X.shape)   # (num_items, 512)
pca2 = PCA(n_components=500, random_state=42)
X_reduced = pca2.fit_transform(X)
df_new["text_emb_pca"] = list(X_reduced)
print("Shape:", X_reduced.shape)

In [ ]:
df_new.to_pickle("final_embeddings2.pkl")

In [ ]:
df_new

,sample_id,catalog_content,image_link,price,clean_text,value,unit,text_emb_combined,clip_emb,img_emb_combined,imagenet_emb,img_emb_combined_reduced,cluster,img_emb_pca,text_emb_pca
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,fl,"[0.03326046, 0.01084006, -0.056023084, 0.08874...","[0.009212236, 0.0026213306, -0.030116022, -0.0...","[0.009212236, 0.0026213306, -0.030116022, -0.0...","[0.03180872, 0.054117072, 0.0008719487, 0.0015...","[0.37894392, 0.23570235, 0.11028024, 0.0579818...",7,"[-0.17400457, 0.007072843, -0.24552792, 0.1888...","[0.30803353, -0.15260652, 0.1687608, -0.017769..."
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Item Name: Salerno Cookies, The Original Butte...",32.00,ounce,"[-0.042282123, 0.03143435, -0.04748076, 0.0682...","[0.014391732, -0.049200386, 0.011009298, 0.054...","[0.014391732, -0.049200386, 0.011009298, 0.054...","[0.0049155937, 0.06572621, 4.4906494e-05, 0.00...","[0.25645456, -0.4132224, -0.15626349, 0.106606...",12,"[0.17496683, 0.11125017, 0.19692834, 0.0854049...","[0.2983607, 0.15823239, -0.13267525, 0.0656291..."
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,ounce,"[-0.03363379, 0.0030081016, 0.0166668, 0.08349...","[-0.010649164, -0.038380288, -0.009885343, 0.0...","[-0.010649164, -0.038380288, -0.009885343, 0.0...","[0.025890373, 0.021702567, 0.010921363, 0.0052...","[0.1649065, -0.10940112, 0.12323196, 0.0211850...",11,"[-0.24622448, 0.13913271, -0.06507938, -0.0605...","[0.1844157, 0.061395295, 0.080886394, 0.152943..."
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,ounce,"[-0.10701239, -0.07945337, 0.012586073, -0.057...","[0.058353223, -0.004353989, 0.049397968, 0.017...","[0.058353223, -0.004353989, 0.049397968, 0.017...","[0.028660946, 0.041502114, 0.021437678, 0.0006...","[-0.17508149, 0.11875085, 0.1648584, 0.1514147...",10,"[0.07123037, 0.26059642, 0.034936316, -0.04879...","[-0.21083117, 0.15530604, 0.11411314, -0.08448..."
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,count,"[0.054222934, 0.07242486, -0.043134164, 0.0518...","[-0.035436884, 0.009175435, -0.022357896, -0.0...","[-0.035436884, 0.009175435, -0.022357896, -0.0...","[0.01593395, 0.060073067, 0.017688613, 0.00177...","[0.18787323, 0.45015624, 0.08781451, -0.069979...",7,"[0.2940553, -0.18923311, -0.02011743, 0.146469...","[0.29232854, -0.19940507, 0.041119426, 0.08562..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,245122,"Item Name: Flock, Chicken Skin Crisps Hattie B...",https://m.media-amazon.com/images/I/71jSwzszMW...,6.29,"Item Name: Flock, Chicken Skin Crisps Hattie B...",2.50,ounce,"[-0.0052951975, 0.01747233, -0.020774389, 0.10...","[0.022693401, -0.037833117, 0.015477225, 0.042...","[0.022693401, -0.037833117, 0.015477225, 0.042...","[0.00941888, 0.059758104, 0.011924964, 0.00099...","[0.03255115, -0.18835111, 0.10423276, -0.03348...",13,"[-0.0073115276, 0.31053263, 0.0032834196, -0.2...","[0.36798605, 0.07267519, 0.080711246, 0.069711..."
9996,211102,Item Name: NY Spice Shop Sanded Cinnamon Drops...,https://m.media-amazon.com/images/I/71RgY-MIaz...,10.49,Item Name: NY Spice Shop Sanded Cinnamon Drops...,16.00,ounce,"[-0.114072844, -0.056503575, 0.06565911, 0.037...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0...","[0.01598184, 0.047277354, 0.009380544, 0.00158...","[0.03845788, -0.089166775, -0.03090444, -0.099...",6,"[-0.13067023, -0.123039275, -0.19294

In [ ]:
df_new['value'].value_counts()

,count
value,
1.0000,937
16.0000,513
12.0000,456
8.0000,307
32.0000,274
...,...
412.8000,1
206.0000,1
0.0988,1


In [ ]:


label = df_new['unit']


le = LabelEncoder()

# Fit and transform
df_new['unit_encoded'] = le.fit_transform(label)


In [ ]:
df_new.to_pickle("final_embeddings2.pkl")

In [ ]:
df_new['price'].dtype

dtype('float64')

In [ ]:
df_new


,sample_id,catalog_content,image_link,price,clean_text,value,unit,text_emb_combined,clip_emb,img_emb_combined,imagenet_emb,img_emb_combined_reduced,cluster,img_emb_pca,text_emb_pca,unit_encoded
0,33127,"Item Name: La Victoria Green Taco Sauce Mild, ...",https://m.media-amazon.com/images/I/51mo8htwTH...,4.89,"Item Name: La Victoria Green Taco Sauce Mild, ...",72.00,fl,"[0.03326046, 0.01084006, -0.056023084, 0.08874...","[0.009212236, 0.0026213306, -0.030116022, -0.0...","[0.009212236, 0.0026213306, -0.030116022, -0.0...","[0.03180872, 0.054117072, 0.0008719487, 0.0015...","[0.37894392, 0.23570235, 0.11028024, 0.0579818...",7,"[-0.17400457, 0.007072843, -0.24552792, 0.1888...","[0.30803353, -0.15260652, 0.1687608, -0.017769...",7
1,198967,"Item Name: Salerno Cookies, The Original Butte...",https://m.media-amazon.com/images/I/71YtriIHAA...,13.12,"Item Name: Salerno Cookies, The Original Butte...",32.00,ounce,"[-0.042282123, 0.03143435, -0.04748076, 0.0682...","[0.014391732, -0.049200386, 0.011009298, 0.054...","[0.014391732, -0.049200386, 0.011009298, 0.054...","[0.0049155937, 0.06572621, 4.4906494e-05, 0.00...","[0.25645456, -0.4132224, -0.15626349, 0.106606...",12,"[0.17496683, 0.11125017, 0.19692834, 0.0854049...","[0.2983607, 0.15823239, -0.13267525, 0.0656291...",19
2,261251,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",https://m.media-amazon.com/images/I/51+PFEe-w-...,1.97,"Item Name: Bear Creek Hearty Soup Bowl, Creamy...",11.40,ounce,"[-0.03363379, 0.0030081016, 0.0166668, 0.08349...","[-0.010649164, -0.038380288, -0.009885343, 0.0...","[-0.010649164, -0.038380288, -0.009885343, 0.0...","[0.025890373, 0.021702567, 0.010921363, 0.0052...","[0.1649065, -0.10940112, 0.12323196, 0.0211850...",11,"[-0.24622448, 0.13913271, -0.06507938, -0.0605...","[0.1844157, 0.061395295, 0.080886394, 0.152943...",19
3,55858,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,https://m.media-amazon.com/images/I/41mu0HAToD...,30.34,Item Name: Judee’s Blue Cheese Powder 11.25 oz...,11.25,ounce,"[-0.10701239, -0.07945337, 0.012586073, -0.057...","[0.058353223, -0.004353989, 0.049397968, 0.017...","[0.058353223, -0.004353989, 0.049397968, 0.017...","[0.028660946, 0.041502114, 0.021437678, 0.0006...","[-0.17508149, 0.11875085, 0.1648584, 0.1514147...",10,"[0.07123037, 0.26059642, 0.034936316, -0.04879...","[-0.21083117, 0.15530604, 0.11411314, -0.08448...",19
4,292686,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",https://m.media-amazon.com/images/I/41sA037+Qv...,66.49,"Item Name: kedem Sherry Cooking Wine, 12.7 Oun...",12.00,count,"[0.054222934, 0.07242486, -0.043134164, 0.0518...","[-0.035436884, 0.009175435, -0.022357896, -0.0...","[-0.035436884, 0.009175435, -0.022357896, -0.0...","[0.01593395, 0.060073067, 0.017688613, 0.00177...","[0.18787323, 0.45015624, 0.08781451, -0.069979...",7,"[0.2940553, -0.18923311, -0.02011743, 0.146469...","[0.29232854, -0.19940507, 0.041119426, 0.08562...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,245122,"Item Name: Flock, Chicken Skin Crisps Hattie B...",https://m.media-amazon.com/images/I/71jSwzszMW...,6.29,"Item Name: Flock, Chicken Skin Crisps Hattie B...",2.50,ounce,"[-0.0052951975, 0.01747233, -0.020774389, 0.10...","[0.022693401, -0.037833117, 0.015477225, 0.042...","[0.022693401, -0.037833117, 0.015477225, 0.042...","[0.00941888, 0.059758104, 0.011924964, 0.00099...","[0.03255115, -0.18835111, 0.10423276, -0.03348...",13,"[-0.0073115276, 0.31053263, 0.0032834196, -0.2...","[0.36798605, 0.07267519, 0.080711246, 0.069711...",19
9996,211102,Item Name: NY Spice Shop Sanded Cinnamon Drops...,https://m.media-amazon.com/images/I/71RgY-MIaz...,10.49,Item Name: NY Spice Shop Sanded Cinnamon Drops...,16.00,ounce,"[-0.114072844, -0.056503575, 0.06565911, 0.037...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0...","[-0.0076368647, 0.013237051, 0.0057303878, 0.0...","[0.01598184, 0.047277354, 0.009380544, 0.00158...","[0.03845788, -0.089166775, -0.03090444, -0.099...",6,"[-0

In [ ]:
# =========================
# 1️⃣ Import Libraries
# =========================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# =========================
# 2️⃣ Load Data
# =========================
# Assume df is already loaded
# df = pd.read_csv('your_file.csv')

# Convert embeddings from list to numpy array
df_new['text_emb_pca'] = df_new['text_emb_pca'].apply(np.array)
df_new['img_emb_combined_reduced'] = df_new['img_emb_combined_reduced'].apply(np.array)
df_new['img_emb_pca'] = df_new['img_emb_pca'].apply(np.array)

# =========================
# 3️⃣ Prepare Features
# =========================
# Combine embeddings and other numeric features
# Example: text_emb + img_emb_pca + unit_encoded (already encoded)
X = np.stack(df_new['text_emb_pca'].values)  # (num_samples, text_emb_dim)
img_X = np.stack(df_new['img_emb_pca'].values)  # (num_samples, img_emb_pca_dim)
unit_X = df_new['unit_encoded'].values.reshape(-1, 1)

# Concatenate all features
X_all = np.concatenate([X, img_X, unit_X], axis=1)

# Target
y = df_new['price'].values

# =========================
# 4️⃣ Train/Test Split
# =========================
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42
)

# =========================
# 5️⃣ Scale Features
# =========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# =========================
# 6️⃣ Neural Network Model
# =========================
def build_nn(input_dim):
    model = Sequential([
        Dense(1000, activation='relu', input_dim=input_dim),
        Dropout(0.2),
        Dense(500, activation='relu'),
        Dropout(0.2),
        Dense(250, activation='relu'),
        Dense(128, activation='relu'),
        Dense(4, activation='relu'),
        Dense(1)  # Regression output
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])
    return model

nn_model = build_nn(X_train_scaled.shape[1])
history = nn_model.fit(
    X_train_scaled, y_train,
    validation_split=0.1,
    epochs=500,
    batch_size=32,
    verbose=0
)

# Predict
y_pred_nn = nn_model.predict(X_test_scaled).flatten()

# Metrics
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

# =========================
# 7️⃣ Random Forest Model
# =========================
rf_model = RandomForestRegressor(
    n_estimators=200, max_depth=10, random_state=42
)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# =========================
# 8️⃣ Ridge Regression Model
# =========================
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_model.predict(X_test_scaled)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

# =========================
# 9️⃣ Compare Results
# =========================
results = pd.DataFrame({
    'Model': ['Neural Network', 'Random Forest', 'Ridge Regression'],
    'MSE': [mse_nn, mse_rf, mse_ridge],
    'R2': [r2_nn, r2_rf, r2_ridge]
})

print(results)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
              Model         MSE        R2
0    Neural Network  891.922200  0.110246
1     Random Forest  975.947947  0.026425
2  Ridge Regression  897.266241  0.104915


In [ ]:


class MultiModalPricePredictor(nn.Module):
    def __init__(self, text_dim=500, img_dim=812, num_tab_features=2, hidden_dim=512, fusion_dim=656):
        super().__init__()

        # Project to a common dimension
        self.text_proj = nn.Linear(text_dim, fusion_dim)
        self.img_proj = nn.Linear(img_dim, fusion_dim)

        # Attention over modalities
        self.attn_fc = nn.Linear(fusion_dim * 2, 2)

        # Fusion + regression layers
        self.fc1 = nn.Linear(fusion_dim + num_tab_features, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        self.out = nn.Linear(hidden_dim // 2, 1)

    def forward(self, text_feat, img_feat, tab_feat):
        # Project to same latent space
        text_feat = self.text_proj(text_feat)
        img_feat = self.img_proj(img_feat)

        # Compute attention weights
        weights = self.attn_fc(torch.cat([text_feat, img_feat], dim=1))
        weights = F.softmax(weights, dim=1)
        w_text, w_img = weights[:, 0:1], weights[:, 1:2]

        # Weighted fusion
        fused = w_text * text_feat + w_img * img_feat

        # Combine fused + tabular
        x = torch.cat([fused, tab_feat], dim=1)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        price = self.out(x)
        return price


In [ ]:

class PriceDataset(Dataset):
    def __init__(self, pkl_path, tabular_cols, target_col="price"):
        self.df = pd.read_pickle(pkl_path)
        self.tabular_cols = tabular_cols
        self.target_col = target_col

        # Convert embeddings from string → numpy
        self.df["text_emb_pca"] = self.df["text_emb_pca"].apply(lambda x: np.array(x, dtype=np.float32))
        self.df["img_emb_pca"] = self.df["img_emb_pca"].apply(lambda x: np.array(x, dtype=np.float32))
        self.df["img_emb_combined_reduced"] = self.df["img_emb_combined_reduced"].apply(lambda x: np.array(x, dtype=np.float32))


    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
      row = self.df.iloc[idx]

      text_feat = torch.tensor(row["text_emb_pca"], dtype=torch.float32)

      # Merge image embeddings safely
      img_feats = []
      for col in ["img_emb_pca", "img_emb_combined_reduced"]:
          if col in self.df.columns and isinstance(row[col], (list, np.ndarray)):
              img_feats.append(np.array(row[col], dtype=np.float32))
      img_feat = torch.tensor(np.concatenate(img_feats) if img_feats else np.zeros(812), dtype=torch.float32)

      # Handle tabular features (convert non-numeric to numeric)
      tab_values = []
      for col in self.tabular_cols:
          val = row[col]
          # convert strings -> numeric encoding
          if isinstance(val, str):
              val = hash(val) % 10_000 / 10_000.0   # small numeric hash
          elif pd.isna(val):
              val = 0.0
          tab_values.append(val)

      tab_feat = torch.tensor(tab_values, dtype=torch.float32)
      target = torch.tensor(row[self.target_col], dtype=torch.float32)

      return text_feat, img_feat, tab_feat, target


In [ ]:


# ===== CONFIG =====
pkl_path = "final_embeddings.pkl"
tabular_cols = ["unit_encoded", "value"]  # replace with your actual tab columns
target_col = "price"
batch_size = 32
epochs = 200
lr = 1e-5
weight_decay = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== DATA =====
dataset = PriceDataset(pkl_path, tabular_cols, target_col)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

num_tab_features = len(tabular_cols)





In [ ]:
model = MultiModalPricePredictor(
    text_dim=500, img_dim=812, num_tab_features=num_tab_features, hidden_dim=512
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
criterion = nn.MSELoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_idx, (text_feat, img_feat, tab_feat, price) in enumerate(
        tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    ):
        text_feat, img_feat, tab_feat, price = (
            text_feat.to(device),
            img_feat.to(device),
            tab_feat.to(device),
            price.to(device),
        )

        optimizer.zero_grad()
        pred = model(text_feat, img_feat, tab_feat)
        loss = criterion(pred.squeeze(), price)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (batch_idx + 1) % 50 == 0:
            print(f"  [Batch {batch_idx+1}] Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {avg_loss:.4f}")
    torch.save(model.state_dict(), f"price_predictor_epoch{epoch+1}.pt")

print("✅ Training complete and model saved.")


Epoch 1/200:  18%|█▊        | 56/313 [00:01<00:05, 43.15it/s]

  [Batch 50] Loss: 1910.3280


Epoch 1/200:  34%|███▍      | 106/313 [00:02<00:04, 43.10it/s]

  [Batch 100] Loss: 1367.6259


Epoch 1/200:  50%|████▉     | 156/313 [00:03<00:03, 42.33it/s]

...

  [Batch 250] Loss: 41.3517


Epoch 200/200:  98%|█████████▊| 308/313 [00:07<00:00, 40.54it/s]

  [Batch 300] Loss: 74.2110


Epoch 200/200: 100%|██████████| 313/313 [00:08<00:00, 39.11it/s]

Epoch 200/200 - Avg Loss: 184.2320
✅ Training complete and model saved.
